In [1]:
from openai import OpenAI
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore

In [4]:
def get_food_nutrients(data):
    food_data = []

    for doc in data:
        food_info = {
            "Food Name": doc.page_content,
            "Nutrients": doc.metadata
        }
        food_data.append(food_info)

    return food_data

def generate_prompt(query, food_data):
    prompt_template = """
    Answer the user's query: {query}

    Using the food data information provided. Not all data will be required to properly answer the user's query. Only use the data for the food which they are asking about. Don't interpolate information.
    For example, if the user asks about potential allergens, answer analytically, using the information available to you to cite the source. You have data from the FDA loaded into your context. So refer
    to the specific nutrients, nutrient types, contents, etc., from the data you have to make sure the user gets the most correct answer.

    Please note that nutrient values are provided per 100 grams of food. Additionally, if a nutrient value is listed as 0.0, it may either indicate that the nutrient is absent or is below the detectable limit based on the analysis methods used.

    The user may input their question about a very specific branded food item, or a more general food item, such as an ingredient. In the case where they ask about a specific branded food item by name, use only the data associated with that food item. In the case where the user asks about a general food, you'll have multiple brands of that food available in your context—choose the most general version of that product in that case.

    If the information provided isn't enough to accurately answer the question, reply with "I'm sorry, I don't have enough information to accurately answer that question."

    Make sure to start the response by mentioning the item you are referring to:

    Food Data: {food_data}
    """
    return prompt_template.format(query=query, food_data=food_data)

def get_completion(query, food_data):
    client = OpenAI()
    prompt = generate_prompt(query, food_data)

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content

def retrieve_food_data_from_pinecone(query):
    embedding_model = PineconeEmbeddings(model="multilingual-e5-large")
    food_data_index = PineconeVectorStore.from_existing_index("food-data", embedding_model)
    retrieved_food_data = food_data_index.similarity_search(query, k=10)
    food_data = get_food_nutrients(retrieved_food_data)

    return food_data



In [5]:
user_query = "give me a breakdown of the nutrients in the cheerios and potential allergens "
retrieved_food_data = retrieve_food_data_from_pinecone(user_query)

response = get_completion(user_query, retrieved_food_data)
print(response)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1073450d0>


### Cheerios Protein Oats and Honey Cereal

#### Nutritional Breakdown per 100 grams:

- **Energy**: 382 KCAL
- **Protein**: 12.73 G
- **Total Lipid (Fat)**: 4.55 G
  - Saturated Fat: 0.91 G
  - Monounsaturated Fat: 1.82 G
  - Polyunsaturated Fat: 1.82 G
  - Trans Fat: 0.0 G
- **Carbohydrates**: 74.55 G
  - Total Sugars: 30.91 G
  - Fiber: 7.3 G
- **Cholesterol**: 0.0 MG

#### Vitamins and Minerals:
- **Calcium**: 182.0 MG
- **Iron**: 11.45 MG
- **Magnesium**: 58.0 MG
- **Phosphorus**: 182.0 MG
- **Potassium**: 273.0 MG
- **Sodium**: 509.0 MG
- **Zinc**: 8.18 MG
- **Vitamin A**: 909.0 IU
- **Vitamin B-6**: 1.091 MG
- **Vitamin B-12**: 2.73 UG
- **Vitamin C**: 10.9 MG
- **Vitamin D**: 73.0 IU
- **Thiamin (Vitamin B1)**: 1.0 MG
- **Riboflavin (Vitamin B2)**: 0.927 MG

#### Potential Allergens:
The data provided does not specify any potential allergens. However, cereals like Cheerios Protein Oats and Honey typically contain ingredients such as oats, wheat, and sometimes soy or nuts, which